In [ ]:
from mriqc_learn.datasets import load_dataset

(train_x, train_y), (_, _) = load_dataset(dataset="SHIP183", split_strategy="none")
train_x["site"] = train_y.site

train_y = train_y[["rating"]].values.squeeze().astype(int)
print(f'Excluded={100 * (train_y < 1).sum() / len(train_y)}')
print(f'Accept={100 * (train_y >= 1).sum() / len(train_y)}')
train_y[train_y >= 1] = 1

Load model 

In [1]:
from joblib import load

# Load the trained model
model = load("/mnt/sda1/Repos/mriqc/mriqc-learn/mriqc-learn/mriqc_learn/data/classifier.joblib")

/home/jaimebarranco/miniconda3/envs/a-eye/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RobustScaler from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jaimebarranco/miniconda3/envs/a-eye/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jaimebarranco/miniconda3/envs/a-eye/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 whe

Load dataset

In [2]:
import pandas as pd

# Load the new dataset
ds_aux = pd.read_csv("/mnt/sda1/Repos/mriqc/mriqc-learn/mriqc-learn/mriqc_learn/datasets/SHIP1210.tsv", sep="\t")
# ds from the 3rd column
ds = ds_aux.iloc[:, 1:] # 2 if dataset contains ratings, 1 if not
# move first column to the last
ds = ds[[c for c in ds if c not in ["site"]] + ["site"]]

Preprocessing

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier as RFC
from mriqc_learn.models import preprocess as pp

# Preprocess the new dataset
preprocessor = Pipeline(
    [
        # (
        #     "drop_ft",
        #     pp.DropColumns(
        #         drop=[f"size_{ax}" for ax in "xyz"] + [f"spacing_{ax}" for ax in "xyz"]
        #     ),
        # ),
        (
            "scale",
            pp.SiteRobustScaler(
                with_centering=True,
                with_scaling=True,
            ),
        ),
        ("site_pred", pp.SiteCorrelationSelector()),
        # ("winnow", pp.NoiseWinnowFeatSelect(use_classifier=True)),
        # ("drop_site", pp.DropColumns(drop=["site"])),
        # (
        #     "rfc",
        #     RFC(
        #         bootstrap=True,
        #         class_weight=None,
        #         criterion="gini",
        #         max_depth=10,
        #         max_features="sqrt",
        #         max_leaf_nodes=None,
        #         min_impurity_decrease=0.0,
        #         min_samples_leaf=10,
        #         min_samples_split=10,
        #         min_weight_fraction_leaf=0.0,
        #         n_estimators=400,
        #         oob_score=True,
        #     ),
        # ),
    ]
)

ds_processed = preprocessor.fit_transform(ds)

Predict

In [4]:
# Predict the classes of the new dataset
y_pred = model.predict(ds_processed)

In [5]:
# add the index of the excluded ones to a list
excluded = []
for i, x in enumerate(y_pred):
    if x == 0:
        excluded.append(i)

In [6]:
# "bids_name" of the indeces in "excluded" in "df_aux"
excluded_bids = [] # bids names
for i in excluded:
    excluded_bids.append(ds_aux.iloc[i, 0])

Reports names of the excluded subjects

In [7]:
import os
import pandas as pd

# Load dataset into a pandas DataFrame 
df = pd.read_csv('/home/jaimebarranco/Desktop/MRI-QC/mriqc/mriqc_non-labeled-dataset/group_T1w.tsv', sep='\t')
df = df.sort_values(by='bids_name') # order by df['bids_name']
df_names = pd.read_csv('/home/jaimebarranco/Desktop/MRI-QC/fetal/fetalqc_non-labeled-dataset/bids_csv.csv', sep=',')
df_names['bids_name'] = df_names['im'].str.split('/').str[-1].str.split('.').str[0]
df_names = df_names.sort_values(by='bids_name') # order by df_names['sub']
df['name'] = df_names['name']
# move last column to second position
cols = list(df)
cols.insert(1, cols.pop(cols.index('name')))
df = df.loc[:, cols]

subs_ls = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset'
files_1 = [(os.path.basename(f)).split('.')[0] for f in sorted(os.listdir(subs_ls)) if not f.startswith('.')]
files_1 = [int(f) for f in files_1] # files from string to int
numbers_1 = [f for f in range(1, len(files_1)+1)]
df_1 = pd.DataFrame({'Subject': files_1, 'Number': numbers_1})

subs = '/home/jaimebarranco/Downloads/samples_v3'
files_2 = [(os.path.basename(f)).split('.')[0] for f in sorted(os.listdir(subs)) if not f.startswith('.')]
files_2 = [int(f) for f in files_2] # files from string to int
numbers_2 = [f for f in range(1, len(files_2)+1)]
df_2 = pd.DataFrame({'Subject': files_2, 'Number': numbers_2})

# df_1 only with df_2 subjects
df_3 = df_1[df_1['Subject'].isin(df_2['Subject'])]
numbers_3 = df_3['Number']
bids_names = [f'sub-{n:03}_T1w' for n in numbers_3]
df_3['bids_name'] = bids_names # add bids names

df_aux = pd.read_excel('/home/jaimebarranco/Desktop/MRI-QC/scores.xlsx', sheet_name='brainmask_avg_data')
df_aux['bids_name'] = bids_names
# move last column to second position
cols = list(df_aux)
cols.insert(1, cols.pop(cols.index('bids_name')))
df_aux = df_aux.loc[:, cols]

# subdataframe of df without the bids_names in df_aux
df = df[~df['bids_name'].isin(df_aux['bids_name'])] # pure testing set

/tmp/ipykernel_97301/3310871597.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['bids_name'] = bids_names # add bids names


In [8]:
import pandas as pd

df1 = pd.read_excel('/home/jaimebarranco/Desktop/MRI-QC/scores.xlsx', sheet_name='brainmask_avg_data') # old ones
dfaux = pd.read_csv('/home/jaimebarranco/Desktop/MRI-QC/mriqc/mriqc_non-labeled-dataset/group_T1w.tsv', sep='\t') # IQMs
df2 = pd.read_excel('/home/jaimebarranco/Desktop/MRI-QC/excluded_edited.xlsx', sheet_name='excluded') # excluded (new ones)
# remove subjects without 'my_rate'
df2_filtered = df2[(~df2['my_rate'].isna()) & (df2['my_rate'] != '')]
# dfaux only with the subjects in df2 in column 'bids_name'
dfaux_filtered = dfaux[dfaux['bids_name'].isin(df2_filtered['bids_name'])]
# reset 'index' column of dfaux_filtered to add ratings
dfaux_filtered = dfaux_filtered.reset_index(drop=True)
# insert 'my_rate' column from df2_filtered to dfaux_filtered
dfaux_filtered['rating'] = df2_filtered['my_rate']
cols = list(dfaux_filtered)
cols.insert(1, cols.pop(cols.index('rating')))
dfaux_filtered = dfaux_filtered.loc[:, cols]

df1_aux = df1.drop(columns=['name','sub','subject','rating','rating_text','blur','blur_text','noise','noise_text','motion','motion_text','bgair','bgair_text'])
df_3 = df_3.reset_index(drop=True)
bids_names = df_3['bids_name']
ratings = df1_aux['exclusion']
df1_filtered = pd.DataFrame({'bids_name': bids_names, 'rating': ratings})
# insert df1_aux into df1_filtered
df1_aux = df1_aux.drop(columns=['exclusion'])
df1_filtered = pd.concat([df1_filtered, df1_aux], axis=1)

# join df1_filtered and dfaux_filtered
df_merged = pd.concat([df1_filtered, dfaux_filtered], ignore_index=True)

# large scale dataset = dfaux - df_merged
df_ls_aux = dfaux.copy()
df_ls_aux['name'] = df_names['name']
df_ls = df_ls_aux[~df_ls_aux['bids_name'].isin(df_merged['bids_name'])]

In [13]:
# reports name of the indeces in "excluded" in "df_ls"
excluded_names = [] # reports names
for i in excluded:
    excluded_names.append(df_ls_aux.iloc[i, list(df_ls_aux.columns).index('name')])

In [14]:
df_excluded = pd.DataFrame({'bids_name': excluded_bids, 'name': excluded_names})
print(f"Excluded subjects: {len(df_excluded)}/{len(y_pred)}")
print(df_excluded)

Excluded subjects: 21/1210
       bids_name       name
0    sub-072_T1w  sub-USOLR
1   sub-1063_T1w  sub-IHOSM
2   sub-1085_T1w  sub-THNQV
3    sub-174_T1w  sub-FYOFI
4    sub-178_T1w  sub-HDMRF
5    sub-237_T1w  sub-KIPSN
6    sub-245_T1w  sub-DPCYK
7    sub-253_T1w  sub-LNDGX
8    sub-274_T1w  sub-DCIUE
9    sub-282_T1w  sub-RUBRL
10   sub-566_T1w  sub-PHYIT
11   sub-626_T1w  sub-IZDXI
12   sub-666_T1w  sub-LQDAH
13   sub-697_T1w  sub-HQITR
14   sub-772_T1w  sub-OJULJ
15   sub-831_T1w  sub-LVKSF
16   sub-881_T1w  sub-UDSIN
17   sub-926_T1w  sub-SIRXJ
18   sub-938_T1w  sub-MIFZE
19   sub-950_T1w  sub-OFJWM
20   sub-952_T1w  sub-VKIMP


Copy subjects' reports to a folder

In [ ]:
import shutil

reports_folder = '/home/jaimebarranco/Desktop/MRI-QC/fetal/fetalqc_non-labeled-dataset'
output_folder = '/home/jaimebarranco/Downloads/excluded_mriqclearn'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# copy html reports from reports_folder that match the subjects in excluded dataframe to output_folder
for i in range(len(excluded)):
    subject = df_excluded['name'].values[i]
    for filename in os.listdir(reports_folder):
        if filename.startswith(f'{subject}_report'):
            shutil.copy(f'{reports_folder}/{filename}', f'{output_folder}/{filename}')